Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

## Important notes about grading:

1. **Compiler errors:** All code you submit must compile. Programs that do not compile will probably receive an automatic zero. If you are having trouble getting your assignment to compile, please visit consulting hours. If you run out of time, it is better to comment out the parts that do not compile, than hand in a more complete file that does not compile.
2. **Late assignments:** Please carefully review the course website's policy on late assignments, as all assignments handed in after the deadline will be considered late. Verify on moodle that you have submitted the correct version, before the deadline. Submitting the incorrect version before the deadline and realizing that you have done so after the deadline will be counted as a late submission.

# Recursive programs 

Let us implement a few facts about natural numbers encoded as church numerals. Use the constant `0` for representing `0` and `s(X)` to represent the successor function.

## Problem 1

Implement the predicate `natural_number` such that the following tests pass. 

In [1]:
natural_number(0).
natural_number(s(N)) :- natural_number(N).

In [2]:
?- natural_number(0) -> true; abort.
?- natural_number(s(s(s(0)))) -> true; abort.
?- not(natural_number(s(s(z)))) -> true; abort.

true.
true.
true.

BTW, the strange syntax above is Prolog's way of writing if-then-else. 

```
if g then t else f
```

gets translated to 

```
g -> t; f
```

where `g` is the goal. We didn't cover this in class and you will not have to worry about this. 

## Problem 2

Implement the predicate 

```prolog
plus(X,Y,Z)
```

which states that `X+Y=Z`. Encode the following axioms for addition:

\\[
\begin{array}{c}
\forall x. plus(x,0,x) \\
\forall x. plus(0,x,x) \\
\forall x,y,z. plus(x,y,z) \rightarrow plus(s(x),y,s(z))
\end{array}
\\]

In [3]:
plus(X,0,X).
plus(0,X,X).
plus(s(X),Y,s(Z)) :- plus(X,Y,Z).

In [4]:
?- plus(s(s(0)),s(s(0)),s(s(s(s(0))))) -> true; abort.
?- plus(s(s(0)),X,s(s(s(s(s(0)))))), X = s(s(s(0))) -> true; abort.
?- not(plus(s(s(0)),X,0)) -> true; abort.
?- plus(X,Y,s(s(s(0)))), X=s(s(0)), Y=s(0) -> true; abort.

true.
X = s(s(s(0))) .
X = _1822 .
Y = s(0), X = s(s(0)) .

## Problem 3

We can represent multiplication using repeated addition. Write a predicate `mult(X,Y,Z)` using `plus(X,Y,Z)` where `mult(X,Y,Z)` represents `X*Y=Z`. Do not use built in arithmetic.

In [5]:
mult(X,0,0).
mult(X,s(Y),Z) :- mult(X,Y,M), plus(M,X,Z).

In [6]:
?- mult(s(s(s(0))),s(s(0)),X), X = s(s(s(s(s(s(0)))))) -> true; abort.
?- mult(s(s(s(0))),0,X), X = 0 -> true; abort.

X = s(s(s(s(s(s(0)))))) .
X = 0 .

## Problem 4

Let's implement a conversion function from Church numerals to Prolog integers. The predicate 

```prolog
to_int(X,Y)
```

holds if `X` is the church numeral that corresponds to the Prolog integer `Y`. See test cases for what is expected.

In [7]:
to_int(0,0).
to_int(s(X),N) :- to_int(X,M), N is M+1.

We now define `of_int` based on `to_int` definition. For example `of_int(1,s(z))`. 

The exclamation mark at the end is the **cut** operator. We will study the cut operator in the future lectures. You will also see more strategically placed cut operators in the tests. You will not need to understand what they are for solving this programming assignment. 

In [8]:
of_int(X,Y) :- to_int(Y,X), !.

In [9]:
?- of_int(3,Y), Y = s(s(s(0))) -> true; abort.
?- to_int(s(s(0)),X), X = 2 -> true; abort.

Y = s(s(s(0))) .
X = 2 .

## Problem 5

Implement subtraction predicate `sub(X,Y,Z)` which holds if `X-Y=Z` where X,Y and Z are church numerals. Do not use built in arithmetic subtraction.

In [10]:
sub(X,X,0).
sub(X,0,X).
sub(s(X),s(Y),Z) :- sub(X,Y,Z).

In [11]:
?- of_int(5,X), of_int(4,Y), sub(X,Y,Z), to_int(Z,Result), Result = 1 -> true; abort.
?- of_int(4,X), of_int(4,Y), sub(X,Y,Z), to_int(Z,Result), Result = 0 -> true; abort.
/* 4-5 is undefined; derives false. */
?- of_int(4,X), of_int(5,Y), not(sub(X,Y,Z)) -> true; abort.

Y = s(s(s(s(0)))), X = s(s(s(s(s(0))))), Z = s(0), Result = 1 .
Y = s(s(s(s(0)))), X = s(s(s(s(0)))), Z = 0, Result = 0 .
Y = s(s(s(s(s(0))))), X = s(s(s(s(0)))), Z = _2176 .

## Problem 6

We can represent division using repeated substraction. Implement `div(X,Y,Z)` which stands for `X/Y=Z`. Do not use built in arithmetic division. You can use arithmetic comparison.

In [12]:
div(Y,Y,s(0)) :- not(sub(Y,0,0)).
div(X,Y,Z) :- not(sub(Y,0,0)),sub(X,Y,R),div(R,Y,M),plus(s(0),M,Z).

In [13]:
?- of_int(10,X), of_int(5,Y), div(X,Y,Z), to_int(Z,Result), Result = 2, ! -> true; abort. 
/* 5/10 is undefined; derives false. */
?- of_int(5,X), of_int(10,Y), not(div(X,Y,Z)) -> true; abort.
/* 5/0 is undefined; derives false. */
?- of_int(5,X), of_int(0,Y), not(div(X,Y,Z)) -> true; abort.

Y = s(s(s(s(s(0))))), X = s(s(s(s(s(s(s(s(s(s(0)))))))))), Z = s(s(0)), Result = 2 .
Y = s(s(s(s(s(s(s(s(s(s(0)))))))))), X = s(s(s(s(s(0))))), Z = _2188 .
Y = 0, X = s(s(s(s(s(0))))), Z = _2176 .

## Problem 7

Implement a predicate `range(S,E,M)` which holds if the integer `M` is within the range `S` to `E` including `S` and `E`. 

In [14]:
range(S,E,M) :- S=<M,E>=M. 

In [15]:
?- not(range(1,2,0)) -> true; abort.
?- range(1,2,1) -> true; abort.
?- range(1,2,2) -> true; abort.
?- not(range(1,2,3)) -> true; abort.

true.
true.
true.
true.

# Programming with Lists

Let's implement some programs over lists.

## Problem 8

Implement reverse of a list using the predicate `reverse(X,RevX)` where `RevX` is the reverse of the list `X`. You might want to use `append`. 

In [16]:
append([],E,[E]).
append([H|T],E,[H|Y]) :- append(T,E,Y).
reverse([],[]).
reverse([H],[H]).
reverse([H|T],X) :- reverse(T,Y),append(Y,H,X).

In [17]:
?- reverse([],X), X = [] -> true; abort.
?- reverse([1,2,3],X), X = [3,2,1] -> true; abort.
?- reverse([A,B,C],X), X = [C,B,A] -> true; abort.

X = [  ] .
X = [ 3, 2, 1 ] .
A = _1866, X = [ _1878, _1872, _1866 ], C = _1878, B = _1872 .

## Problem 9

Implement list membership predicate `member(X,L)` which holds if $X \in L$.

In [18]:
member(X,[X|T]).
member(X,[Y|T]):- member(X,T).

In [19]:
?- member(1,[1,2,3]) -> true; abort.
?- not(member(4,[1,2,3])) -> true; abort.

true.
true.

## Problem 10

Implement `partition(L,P,S)` such that 

* `P` is the prefix of `L` and
* `S` is the suffix of `L` and
* `append(P,S,L)` holds
* If `L` is `[]`, then `P` and `S` are `[]`.
* If `L` is `[H]`, then `P` is `[H]` and `S` is `[]`. 
* Otherwise, 
  * let length of `L` be `N`. Then length of `P` is `div(N/2)`. Use Prolog's [built-in integer division](https://www.swi-prolog.org/pldoc/man?function=div/2).
  * length of `S` is `N - div(N/2)`. 

You may need to redefine `len`,`prefix`,`suffix`,`append` predicates that we have seen in class. You may also need to use Prolog comparison operator `>=` or `>` depending on how you write the predicate. 

In [20]:
append2([],E,E).
append2([H|T],E,[H|Y]) :- append2(T,E,Y).

len2([],0).
len2([H|T],X) :- len2(T,Y),X is Y+1.

partition([],[],[]).
partition([H],[H],[]).
partition(L,P,S) :- len2(L,N),Plen is div(N,2),len2(P,Plen), Slen is N - Plen, len2(S,Slen),append2(P,S,L).

In [21]:
?- partition([],[],[]) -> true; abort.
?- partition([H],[H],[]) -> true; abort.
?- partition([1,2,3],[1],[2,3]) -> true; abort.

true.
H = _1806 .
true.

## Problem 11

Implement the predicate `merge(X,Y,Z)` where `X` and `Y` are sorted, and `Z` contains the same elements as `U` where `append(X,Y,U)` but `Z` is also additionally sorted.

In [22]:
merge([],[],[]).
merge([],X,X).
merge(X,[],X).
merge([H1|T1],[H2|T2],[H1|T3]) :- H1=<H2,merge(T1,[H2|T2],T3).
merge([H1|T1],[H2|T2],[H2|T3]) :- H2<H1,merge([H1|T1],T2,T3).

In [23]:
?- merge([],[1],[1]) -> true; abort.
?- merge([1],[],[1]) -> true; abort.
?- merge([1,3,5],[2,4,6],[1,2,3,4,5,6]) -> true; abort.

true.
true.
true.

## Problem 12

Implement predicate `mergesort(L,SL)` where `SL` is the sorted version of `L`. Use the predicate to partition the list `L` into two, sort each on separately (using `mergesort`) and combine the individual sorted list using `merge`. 

In [24]:
mergesort([],[]).
mergesort([H],[H]).
mergesort(L,SL) :- partition(L,P,S), mergesort(P,Pnice),mergesort(S,Snice),merge(Pnice,Snice,SL).

In [25]:
?- mergesort([3,2,1],[1,2,3]), ! -> true; abort.
?- mergesort([1,2,3],[1,2,3]), ! -> true; abort.
?- mergesort([],[]), ! -> true; abort.

true.
true.
true.